In [427]:
import numpy as np
import pandas as pd
import seaborn as sns
import pickle
import scipy
from scipy.sparse import csc_matrix
from collections import defaultdict
from os import listdir
import matplotlib.pyplot as plt
import sys
%matplotlib inline

In [281]:
with open('./../PickleFiles/num_to_ind_shot.pkl','r') as pickleFile:
    num_to_ind = pickle.load(pickleFile)

with open('./../PickleFiles/ind_to_num_shot.pkl','r') as pickleFile:
    ind_to_num = pickle.load(pickleFile)
    
with open('./../PickleFiles/player_names.pkl','r') as pickleFile:
    num_to_name = pickle.load(pickleFile)

with open('./../PickleFiles/tourn_order.pkl','r') as pickleFile:
    tourn_order = pickle.load(pickleFile)

name_to_num = {value:key for key,value in num_to_name.iteritems()}

In [282]:
rdata = pd.read_csv('./../data/round.csv')

In [284]:
cols = ['Year','Permanent_Tournament_#']
data = pd.concat([pd.read_csv('./../data/%d.csv' % (year))[cols] for year in range(2003,2017)])

In [287]:
data = pd.concat([data[(data.Year==year) & (data['Permanent_Tournament_#']==tourn)] 
                  for year,tourn in tourn_order])
tups = data.drop_duplicates(['Year','Permanent_Tournament_#'])[['Year','Permanent_Tournament_#']].values.tolist()

In [289]:
tournament_groups = {tuple(tup):u/4 for u,tup in enumerate(tups)}

In [349]:
cats = ['tee3','tee45','green0','green5','green10','green20','rough0','rough90',
        'rough375','fairway0','fairway300','fairway540','bunker','other']

Selecting most predictive for first 100 tournament groups

In [425]:
mypath = './../ranks/'
corrs = defaultdict(list)
for path in [f for f in listdir(mypath)]:
    if len([f for f in listdir(mypath+path)])!=28:
        continue
    ratings_adam = {}
    for cat in cats:
        ratings_adam[cat] = np.load('%s%s/%s_ranks.npy' % (mypath,path,cat))
    preds,outcomes = defaultdict(list),defaultdict(list)
    for year,tourn in tups:
        tournament_group = tournament_groups[(year,tourn)]
        if tournament_group==0 or tournament_group>99:
            continue
        rtourn = rdata[(rdata.Tournament_Year==year) & (rdata['Permanent_Tournament_#']==tourn)]
        players = pd.unique(rtourn.Player_Number)
        finishing_positions = rtourn.groupby('Player_Number').Finishing_Position.mean().to_dict()
        finishing_positions = np.array([finishing_positions[player] for player in players])
        players = [num_to_ind[player] for player in players]
        for cat in cats:
            ratings = ratings_adam[cat][players,tournament_group-1]
            nulls = np.isnan(ratings)
            if np.all(nulls):
                continue
            ranking = pd.Series(ratings[~nulls]).rank(ascending=False,pct=True)
            outcome = pd.Series(finishing_positions[~nulls]).rank(pct=True)
            preds[cat].extend(ranking.tolist())
            outcomes[cat].extend(outcome.tolist())
    for cat in cats:
        corrs[cat].extend([(path,np.corrcoef(preds[cat],outcomes[cat])[0,1])])
ratings_adam = {}
for cat in corrs:
    print cat
    for params,corr in sorted(corrs[cat],key=lambda x: x[1],reverse=True):
        print params,corr
    best = sorted(corrs[cat],key=lambda x: x[1],reverse=True)[0][0]
    ratings_adam[cat] = np.load('%s%s/%s_ranks.npy' % (mypath,best,cat))

green5
ranks-300-0.23-0.15-0.7-0.99-15 0.0334430619525
ranks-300-0.23-0.15-0.7-0.9-15 0.0330495874756
ranks-300-0.23-0.15-0.7-0.99-11 0.0316255338421
ranks-300-0.23-0.15-0.7-0.9-11 0.0312755500015
ranks-300-0.23-0.15-0.95-0.9-11 0.0311331558931
ranks-300-0.23-0.15-0.7-0.99-5 0.0272173219901
ranks-300-0.23-0.15-0.7-0.9-5 0.0267517376648
ranks-300-0.23-0.15-0.95-0.9-5 0.0262980901376
fairway300
ranks-300-0.23-0.15-0.7-0.99-15 0.102184271317
ranks-300-0.23-0.15-0.7-0.99-11 0.101737340072
ranks-300-0.23-0.15-0.7-0.9-15 0.101072419563
ranks-300-0.23-0.15-0.7-0.9-11 0.10058957556
ranks-300-0.23-0.15-0.95-0.9-11 0.100391622853
ranks-300-0.23-0.15-0.7-0.99-5 0.0971698408717
ranks-300-0.23-0.15-0.7-0.9-5 0.0951611992448
ranks-300-0.23-0.15-0.95-0.9-5 0.095132283833
green0
ranks-300-0.23-0.15-0.7-0.99-15 0.0537386451518
ranks-300-0.23-0.15-0.7-0.9-15 0.0527338438623
ranks-300-0.23-0.15-0.7-0.99-11 0.0526118776668
ranks-300-0.23-0.15-0.7-0.9-11 0.0518115259237
ranks-300-0.23-0.15-0.95-0.9-11 0.05

In [393]:
mypath = './../Broadie_Aves'
corrs = defaultdict(list)
for BETA in range(1,20):
    ratings_broadie = {}
    for cat in cats:
        ratings_broadie[cat] = np.load('%s/%s_%d.npy' % (mypath,cat,BETA))
    preds,outcomes = defaultdict(list),defaultdict(list)
    for year,tourn in tups:
        tournament_group = tournament_groups[(year,tourn)]
        if tournament_group==0 or tournament_group>99:
            continue
        rtourn = rdata[(rdata.Tournament_Year==year) & (rdata['Permanent_Tournament_#']==tourn)]
        players = pd.unique(rtourn.Player_Number)
        finishing_positions = rtourn.groupby('Player_Number').Finishing_Position.mean().to_dict()
        finishing_positions = np.array([finishing_positions[player] for player in players])
        players = [num_to_ind[player] for player in players]
        for cat in cats:
            ratings = ratings_broadie[cat][players,tournament_group]
            nulls = np.isnan(ratings)
            if np.all(nulls):
                continue
            ranking = pd.Series(ratings[~nulls]).rank(ascending=False,pct=True)
            outcome = pd.Series(finishing_positions[~nulls]).rank(pct=True)
            preds[cat].extend(ranking.tolist())
            outcomes[cat].extend(outcome.tolist())
    for cat in cats:
        corrs[cat].extend([(BETA,np.corrcoef(preds[cat],outcomes[cat])[0,1])])
ratings_broadie = {}
for cat in corrs:
    print cat
    for BETA,corr in sorted(corrs[cat],key=lambda x: x[1],reverse=True):
        print BETA,corr
    best = sorted(corrs[cat],key=lambda x: x[1],reverse=True)[0][0]
    ratings_broadie[cat] = np.load('%s/%s_%d.npy' % (mypath,cat,best))

green5
19 0.0288589193728
18 0.0288485626466
17 0.0287132252283
15 0.0286287490809
16 0.0285791329295
14 0.0283538633723
2 0.0282165508329
13 0.0279845840889
12 0.0277616807094
11 0.0274603423507
3 0.0274084978409
4 0.0273575024845
5 0.0271761371559
10 0.0270247193877
8 0.026507341948
9 0.0264914665114
1 0.0264244161455
6 0.0263673604528
7 0.0262673211156
fairway300
17 0.0939205573528
19 0.0938424312164
18 0.0938331474799
16 0.0936832978931
15 0.0935206092899
14 0.0932079050342
13 0.0928255606266
12 0.0925414197381
11 0.0919883581361
10 0.0914872370664
9 0.0905970792068
8 0.089130144833
7 0.0881685140974
6 0.08635161983
5 0.0842699407938
4 0.0814193009305
3 0.0780283773123
2 0.0739598925037
1 0.0658136949907
green0
18 0.0425585794838
19 0.0425052253607
17 0.0423223339715
16 0.0422779261872
15 0.0420075464944
14 0.0416222989599
13 0.0413495077644
12 0.0410813403561
11 0.0405285882445
10 0.0394654651374
9 0.0384882781383
8 0.0374793767199
7 0.0367915890976
6 0.0355528896185
5 0.034116974

In [394]:
preds_a,preds_b,outcomes = defaultdict(list),defaultdict(list),defaultdict(list)
for year,tourn in tups:
    tournament_group = tournament_groups[(year,tourn)]
    if tournament_group<100:
        continue
    rtourn = rdata[(rdata.Tournament_Year==year) & (rdata['Permanent_Tournament_#']==tourn)]
    players = pd.unique(rtourn.Player_Number)
    finishing_positions = rtourn.groupby('Player_Number').Finishing_Position.mean().to_dict()
    finishing_positions = np.array([finishing_positions[player] for player in players])
    players = [num_to_ind[player] for player in players]
    for cat in cats:
        ratings_a = ratings_adam[cat][players,tournament_group-1]
        ratings_b = ratings_broadie[cat][players,tournament_group]
        nulls = np.logical_or(np.isnan(ratings_a),np.isnan(ratings_b))
        if np.all(nulls):
            continue
        rankings_a = pd.Series(ratings_a[~nulls]).rank(ascending=False,pct=True)
        rankings_b = pd.Series(ratings_b[~nulls]).rank(ascending=False,pct=True)
        outcome = pd.Series(finishing_positions[~nulls]).rank(pct=True)
        preds_a[cat].extend(rankings_a.tolist())
        preds_b[cat].extend(rankings_b.tolist())
        outcomes[cat].extend(outcome.tolist())

In [395]:
corrs = {cat:np.corrcoef(preds_b[cat],outcomes[cat])[0,1] for cat in cats}

In [396]:
ps_and_os = {}
for cat in cats:
    ps_and_os[cat] = pd.DataFrame({'Predictions_Adam':preds_a[cat],
                                   'Predictions_Broadie':preds_b[cat],
                                   'Outcome':outcomes[cat]})

In [397]:
for cat in cats:
    print
    print cat
    print len(preds_a[cat])
    assert len(preds_a[cat])==len(preds_b[cat])
    print 'Adam Corr = ' + str(np.corrcoef(preds_a[cat],outcomes[cat])[0,1])
    print 'Broadie Corr = ' + str(np.corrcoef(preds_b[cat],outcomes[cat])[0,1])
    print 


tee3
18405
Adam Corr = 0.0938411577487
Broadie Corr = 0.0833679270716


tee45
18390
Adam Corr = 0.1159149409
Broadie Corr = 0.103134531138


green0
18405
Adam Corr = 0.0423530577856
Broadie Corr = 0.0329281050352


green5
18405
Adam Corr = 0.028410218751
Broadie Corr = 0.0221749454647


green10
18405
Adam Corr = 0.0381620470831
Broadie Corr = 0.0321626161936


green20
18405
Adam Corr = 0.0422785973426
Broadie Corr = 0.0371818956823


rough0
18395
Adam Corr = 0.0471186933827
Broadie Corr = 0.0386515538089


rough90
18387
Adam Corr = 0.0545953184484
Broadie Corr = 0.0531930897495


rough375
18405
Adam Corr = 0.0885846060155
Broadie Corr = 0.0764562383339


fairway0
18402
Adam Corr = 0.0669607072958
Broadie Corr = 0.0671155323319


fairway300
18404
Adam Corr = 0.078459546099
Broadie Corr = 0.0728461572007


fairway540
18403
Adam Corr = 0.0927115650081
Broadie Corr = 0.0903406071958


bunker
18403
Adam Corr = 0.0411489159762
Broadie Corr = 0.0378856423603


other
17811
Adam Corr = 0.01819

In [398]:
def confidence_interval(df,its):
    corrs_adam,corrs_broadie = [],[]
    for _ in range(its):
        df_ = df.sample(frac=1,replace=True)
        corrs_adam.append(np.corrcoef(df_.Predictions_Adam,df_.Outcome)[0,1])
        corrs_broadie.append(np.corrcoef(df_.Predictions_Broadie,df_.Outcome)[0,1])
    return corrs_adam,corrs_broadie

In [401]:
for cat in cats:
    corrs_adam,corrs_broadie = confidence_interval(ps_and_os[cat],1000)
    print cat
    print 'adam',np.mean(corrs_adam),(np.mean(corrs_adam)-np.std(corrs_adam),np.mean(corrs_adam)+np.std(corrs_adam))
    print 'broadie',np.mean(corrs_broadie),(np.mean(corrs_broadie)-np.std(corrs_broadie),np.mean(corrs_broadie)+np.std(corrs_broadie))

tee3
adam 0.0939576727035 (0.086740202231251604, 0.10117514317566599)
broadie 0.0834652277636 (0.076201273603489189, 0.090729181923627888)
tee45
adam 0.116106498764 (0.1089630968251695, 0.12324990070377417)
broadie 0.103276036556 (0.096144070998337564, 0.11040800211300801)
green0
adam 0.041979481393 (0.034553818217173342, 0.049405144568924995)
broadie 0.0325837886092 (0.025190825913637551, 0.039976751304765419)
green5
adam 0.0286130462772 (0.021410112165255995, 0.035815980389047208)
broadie 0.0224400712976 (0.015224777322259485, 0.029655365272930383)
green10
adam 0.0379346616674 (0.030478584350139656, 0.045390738984562445)
broadie 0.0319458461408 (0.024518226974119949, 0.039373465307543563)
green20
adam 0.0424270925505 (0.035131961466046906, 0.0497222236349344)
broadie 0.0373697374293 (0.029985978772939549, 0.044753496085643642)
rough0
adam 0.0467272725104 (0.039636709108014083, 0.05381783591283585)
broadie 0.0382953500461 (0.031113077670504245, 0.045477622421781265)
rough90
adam 0.054

In [390]:
def confidence_interval(df,its):
    mse_adam,mse_broadie = [],[]
    for _ in range(its):
        df_ = df.sample(frac=1,replace=True)
        mse_adam.append(np.mean((df_.Predictions_Adam - df_.Outcome)**2))
        mse_broadie.append(np.mean((df_.Predictions_Broadie - df_.Outcome)**2))
    return mse_adam,mse_broadie

In [392]:
for cat in cats:
    mse_adam,mse_broadie = confidence_interval(ps_and_os[cat],1000)
    print cat
    print 'adam',np.mean(mse_adam),(np.percentile(mse_adam,q=[2.5,97.5]))
    print 'broadie',np.mean(mse_broadie),(np.percentile(mse_broadie,q=[2.5,97.5]))
    print

tee3
adam 0.150758122285 [ 0.14818405  0.15347676]
broadie 0.152516583167 [ 0.14982694  0.1552522 ]

tee45
adam 0.147090675929 [ 0.14452529  0.14973509]
broadie 0.149208146857 [ 0.14651548  0.15180842]

green0
adam 0.159222851333 [ 0.15645234  0.16188929]
broadie 0.160781717407 [ 0.15809962  0.16348765]

green5
adam 0.161516648996 [ 0.15887383  0.16439285]
broadie 0.162559040779 [ 0.1599492   0.16533102]

green10
adam 0.159984456403 [ 0.15703494  0.16286202]
broadie 0.161033885899 [ 0.15815954  0.16391674]

green20
adam 0.159292029973 [ 0.1566673   0.16195843]
broadie 0.16017710579 [ 0.15760837  0.16307391]

rough0
adam 0.158520224174 [ 0.15583585  0.16159917]
broadie 0.159889384472 [ 0.15696912  0.16275987]

rough90
adam 0.157236574716 [ 0.15455557  0.15988823]
broadie 0.157408471253 [ 0.15459471  0.16027504]

rough375
adam 0.151623859118 [ 0.14908815  0.15428064]
broadie 0.153593646715 [ 0.15078153  0.1563227 ]

fairway0
adam 0.155111882721 [ 0.15258169  0.15763184]
broadie 0.1551118

In [428]:
sys.path.insert(0, '/home/ubuntu/CorrelationStats/')
import corrstats

In [429]:
for cat in cats:
    xy = np.corrcoef(ps_and_os[cat].Outcome,ps_and_os[cat].Predictions_Adam)[0,1]
    xz = np.corrcoef(ps_and_os[cat].Outcome,ps_and_os[cat].Predictions_Broadie)[0,1]
    yz = np.corrcoef(ps_and_os[cat].Predictions_Adam,ps_and_os[cat].Predictions_Broadie)[0,1]
    print cat
    print xy,xz,yz
    print corrstats.dependent_corr(xy,xz,yz,len(ps_and_os[cat]),
                                    twotailed=True, conf_level=0.95, method='steiger')
    print corrstats.dependent_corr(xy,xz,yz,len(ps_and_os[cat]),
                                    twotailed=True, conf_level=0.95, method='zou')
    print

tee3
0.0938411577487 0.0833679270716 0.945386365236
(4.3184313190378996, 1.5796430900216762e-05)
(0.0057176116104975779, 0.015228543973237564)

tee45
0.1159149409 0.103134531138 0.915038376024
(4.2327040024713476, 2.3201832496377861e-05)
(0.0068597798098484607, 0.01870056398105174)

green0
0.0423530577856 0.0329281050352 0.961791392984
(4.6310853689850617, 3.6624189367895355e-06)
(0.0054339457792485094, 0.013415496913948877)

green5
0.028410218751 0.0221749454647 0.925383998651
(2.190573261854333, 0.02849518798816475)
(0.00065589576074724133, 0.011814165463205479)

green10
0.0381620470831 0.0321626161936 0.897498718553
(1.7988026683820473, 0.07206628070941079)
(-0.00053783301010462432, 0.012536155529812194)

green20
0.0422785973426 0.0371818956823 0.886605359826
(1.4531079570405485, 0.14621083390554834)
(-0.0017778837266951182, 0.011970809050482326)

rough0
0.0471186933827 0.0386515538089 0.793047806969
(1.7868167343371046, 0.07398357789036103)
(-0.00082097754033638032, 0.0177541606602